In [1]:
#Importamos las librerias a usar
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

In [2]:
#Leemos nuestros DataFrames a utilizar
df = pd.read_csv('Pokemon.csv')
combats = pd.read_csv('Combat1.csv')
test = pd.read_csv('tests.csv')

In [3]:
test3 = test.copy()

In [5]:
#Reasignamos el nombre a la columna # para facil manipulacion
df.rename(columns={'#':'Num'}, inplace=True)

In [6]:
#Re Asignamos el index para el correcto manejo de la base
df.set_index(df.Num, inplace=True)

In [7]:
#Creamos un nuevo DataSet con las columnas que vamos a usar
poke = df.filter(["Name","HP","Attack","Defense","Sp.Atk","Sp.Def","Speed"], axis=1)
poke.head()

,Name,HP,Attack,Defense,Sp.Atk,Sp.Def,Speed
Num,,,,,,,
1,Bulbasaur,45,49,49,65,65,45
2,Ivysaur,60,62,63,80,80,60
3,Venusaur,80,82,83,100,100,80
4,Charmander,39,52,43,60,50,65
5,Charmeleon,58,64,58,80,65,80


In [8]:
#Remplazamos los numeros con el nombre del pokemon correspondiente
cols = ["First_pokemon","Second_pokemon","Winner"]
fights = combats[cols].replace(df.Name)
fights.head()

,First_pokemon,Second_pokemon,Winner
0,Larvitar,Nuzleaf,Nuzleaf
1,Virizion,Terrakion,Terrakion
2,Togetic,Beheeyem,Beheeyem
3,Slugma,Druddigon,Druddigon
4,Omastar,Shuckle,Omastar


In [9]:
#Transformamos en boleano nuestro resultado del combate
combats.Winner[combats.Winner == combats.First_pokemon] = 0
combats.Winner[combats.Winner == combats.Second_pokemon] = 1

In [10]:
#Creamos una funcion de normalizacion para los datos que usaremos
#Fue robada de internet y modificada por su servidor
def norm(data_df):
    stats=["HP","Attack","Defense","Sp.Atk","Sp.Def","Speed"]
    stats_df=poke[stats].T.to_dict("list")
    first=data_df.First_pokemon.map(stats_df)
    second=data_df.Second_pokemon.map(stats_df)
    lista=[]
    for i in range(len(first)):
        lista.append(np.array(first[i])-np.array(second[i]))
    new_test = pd.DataFrame(lista, columns=stats)
    for c in stats:
        description=new_test[c].describe()
        new_test[c]=(new_test[c]-description['min'])/(description['max']-description['min'])
    return new_test

In [11]:
#Aplicamos la funcion y creamos un nuevo DataSet
#Concatenamos las dos tablas
data = norm(combats)
data = pd.concat([data, combats.Winner], axis=1)

In [12]:
#Usamos la columna Winner para hacer nuestro testeo
X=data.drop("Winner",axis=1)
y=data["Winner"]

In [13]:
#Definimos el train y asignamos el size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [14]:
#Aplicamos modelo de ML para predecir (en el testeo original se usaron 5 y este fue el que mas me agrado)
clf_rfc = RandomForestClassifier(n_estimators=50)
clf_rfc.fit(X_train, y_train)
accuracies = cross_val_score(estimator = clf_rfc, X = X_train, y = y_train, cv = 5,verbose = 1)
y_pred = clf_rfc.predict(X_test)

print('')
print('####### RandomForestClassifier #######')
print('Score : %.4f' % clf_rfc.score(X_test, y_test))
print(accuracies)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse)
print('R2     : %0.2f ' % r2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



####### RandomForestClassifier #######
Score : 0.9466
[0.94760227 0.95173893 0.94928757 0.9501992  0.94881226]

MSE    : 0.05 
MAE    : 0.05 
RMSE   : 0.23 
R2     : 0.79 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.4s finished


In [15]:
#Aplicamos modelo de ML a nuestro test para predecir las victorias
#battle=norm(test)
#pred=clf_rfc.predict(battle)
#test["Winner"]=[test["First_pokemon"][i] if pred[i]==0 else test["Second_pokemon"][i] for i in range(len(pred))]

In [16]:
#Remplazamos los numeros por nombres eh imprimimos los resultados de las batallas
#final = test[cols].replace(poke.Name)
#final.head()

,First_pokemon,Second_pokemon,Winner
0,Magikarp,Seadra,Seadra
1,Diggersby,Qwilfish,Qwilfish
2,Goodra,Kangaskhan,Kangaskhan
3,Quagsire,Stunfisk,Quagsire
4,Sandshrew,Froakie,Froakie


In [35]:
print('Pick a First Pokemon Betwen 1-809')
First = int(input())
print('Pick a Second Pokemon Betwen 1-809')
Second = int(input())

Pick a First Pokemon Betwen 1-809
249
Pick a Second Pokemon Betwen 1-809
150


In [36]:
simple_list=[[First, Second]]
test2=pd.DataFrame(simple_list,columns=['First_pokemon','Second_pokemon'])
test4 = test3.append(test2, ignore_index=True)
battle2=norm(test4)
pred=clf_rfc.predict(battle2)
test4["Winner"]=[test4["First_pokemon"][i] if pred[i]==0 else test4["Second_pokemon"][i] for i in range(len(pred))]
final1 = test4[cols].replace(poke.Name)
final1.tail(1)

,First_pokemon,Second_pokemon,Winner
10000,Lugia,Mewtwo,Mewtwo
